In [ ]:
# install the libraries to load transformers==4.42.4 models
!pip install transformers==4.42.4

In [ ]:
# Import necessary libraries for data manipulation and analysis
import pandas as pd
import numpy as np

# Import visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt

# Import modules from scikit-learn for machine learning tasks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score, classification_report
from sklearn.metrics import f1_score
# Import TensorFlow for deep learning tasks
import tensorflow as tf

import re
import json

In [ ]:
# Import BertTokenizer, TFBertForSequenceClassification from the Hugging Face transformers library
from transformers import BertTokenizer, TFBertForSequenceClassification


In [ ]:
# Set the seed for the TensorFlow random number generator to ensure reproducibility
tf.random.set_seed(42)

###**Data preprocessing for Bert Fine Tunning**

In [ ]:
import pandas as pd
# Load a CSV File containing Dataset of 500 products, narrative and summary (summary of narrative)
data= pd.read_csv("/content/Complains_classification.csv")

In [ ]:
Bert_data = data[['product','narrative']]

In [ ]:
# Creating dependent and independent variables from Bert_data
train_test= Bert_data['narrative']
y = Bert_data['product']
# Further split the temporary set into train (80%) and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(train_test, y, test_size=0.20, stratify=y, random_state=42)

In [ ]:
# Label Encoding
encoder = LabelEncoder()

# fit the encoder to the training labels
y_train_enc = encoder.fit_transform(y_train)

# applying the encoder mapping from training labels to test labels
y_test_enc = encoder.transform(y_test)

###**Tokenization**

In [ ]:
# loading and creating an instance of the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# specifying the maximum length of the input 512
max_length = 512

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
X_train_tokenized = tokenizer(
    X_train.values.tolist(),    # passing the data as a list to the tokenizer
    max_length=max_length,    # specifies the maximum length of the tokenized data
    padding='max_length',    # padding the data to the specified maximum length
    truncation=True,    # truncating the input if it is longer than the specified maximum length
    return_attention_mask=True,    # specifying to return attention masks
    return_tensors='tf',    # specifying to return the output as tensorflow tensors
)
X_test_tokenized = tokenizer(
    X_test.values.tolist(),
    max_length=max_length,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='tf',
)

###**Creating Tensorflow dataset**

In [ ]:
# defining the size of the batches
batch_size = 8

# convert the tokenized input and the output into a batched tensorflow dataset for training
train_tokenized_tf =  tf.data.Dataset.from_tensor_slices((
    dict(X_train_tokenized),
    y_train_enc
 )).batch(batch_size)


# convert the tokenized input and the output into a batched tensorflow dataset for testing
test_tokenized_tf = tf.data.Dataset.from_tensor_slices((
    dict(X_test_tokenized),
    y_test_enc
 )).batch(batch_size)

###**Evaluating the base model's performance in product classification.**

In [ ]:
def bert_f1_score(actual_val,perdicted_val):
    micro_f1_score = f1_score(actual_val, preds_test, average="micro")
    return micro_f1_score

In [ ]:
# Actual product class
actual_val = np.concatenate([y for x, y in test_tokenized_tf], axis=0)

In [ ]:
num_classes = y.nunique()
# Initialize Model using BERT for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Make prediction on test_tokenized_tf
preds_raw_test = model.predict(test_tokenized_tf)
preds_test = np.argmax(np.array(tf.nn.softmax(preds_raw_test.logits)), axis=1)

13/13 [==============================] - 9s 131ms/step


In [ ]:
# Evaluate bert base model
f1_score= bert_f1_score(actual_val, preds_test)
print(f1_score)

0.05000000000000001


###**Fine-Tuning Bert Model on training set**

In [ ]:
num_classes = y.nunique()
# Model initialization using BERT for sequence classification
model =  TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# setting the learning rate for the optimizer
learning_rate = 1e-5

# Setting the optimizer to Adam
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)

# Specify the loss function for the model
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Define evaluation metric(s) for the model
metric = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]

# Compile the model with the chosen optimizer, loss function, and metrics
model.compile(optimizer=optimizer, loss=loss, metrics=metric)

In [ ]:
# Calculate class weights for imbalanced dataset
cw = (y_train_enc.shape[0]) / np.bincount(y_train_enc)

# Create a dictionary mapping class indices to their respective class weights
cw_dict = {}
for i in range(cw.shape[0]):
    cw_dict[encoder.transform(encoder.classes_)[i]] = cw[i]

In [ ]:
# Number of training epochs
n_epochs = 1
#train bert model
bert_base_tuned = model.fit(train_tokenized_tf, epochs=n_epochs, class_weight=cw_dict)

50/50 [==============================] - 56s 420ms/step - loss: 7.8732 - accuracy: 0.5450


###**Evaluating the trained model performance**

In [ ]:
# Generate raw predictions on the test dataset using the trained model
preds_raw_val =  model.predict(test_tokenized_tf)

# Extract predicted labels by finding the index with the highest probability for each example
preds_val = np.argmax(np.array(tf.nn.softmax(preds_raw_val.logits)), axis=1)

13/13 [==============================] - 5s 132ms/step


In [ ]:
preds_raw_val = model.predict(test_tokenized_tf)
print(preds_raw_val)

13/13 [==============================] - 2s 132ms/step
TFSequenceClassifierOutput(loss=None, logits=array([[ 2.27404952e-01, -1.78917751e-01, -1.34568825e-01,
         1.65606901e-01, -1.64335504e-01],
       [-2.35994145e-01,  9.05219197e-01, -3.09063435e-01,
        -1.08652949e-01, -3.39790553e-01],
       [-6.08931342e-03, -6.02557734e-02, -2.49799356e-01,
         6.91297725e-02, -2.67835379e-01],
       [-2.83773422e-01,  8.92094970e-01, -2.87677050e-01,
        -1.60558313e-01, -3.06447208e-01],
       [-2.35994145e-01,  9.05219197e-01, -3.09063435e-01,
        -1.08652949e-01, -3.39790553e-01],
       [-2.35994145e-01,  9.05219197e-01, -3.09063435e-01,
        -1.08652949e-01, -3.39790553e-01],
       [ 2.92592376e-01, -3.03705841e-01, -1.37408972e-01,
         2.73716360e-01, -1.59437269e-01],
       [-2.83773422e-01,  8.92094970e-01, -2.87677050e-01,
        -1.60558313e-01, -3.06447208e-01],
       [-2.60354191e-01,  8.97613049e-01, -2.93567389e-01,
        -1.26155481e-01, 

In [ ]:
actual_val = np.concatenate([y for x, y in test_tokenized_tf], axis=0)
print(actual_val)

[1 1 1 1 1 1 3 1 1 1 2 1 1 2 1 3 1 4 1 1 1 1 1 1 1 1 1 1 4 3 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 3 0 1 1 1 1 3 1 1 1 1 1 1 0 1 1 2 4 1 1 1 1 1 1 2 1 4
 1 1 1 1 0 1 1 1 2 1 1 0 2 3 1 1 1 1 1 3 1 1 1 1 1 0]


In [ ]:
from sklearn.metrics import f1_score
import numpy as np
# Evaluate bert trained model
#f1_score= bert_f1_score(actual_val, preds_val)
#print(f1_score)
# Define the function to calculate F1 score
def bert_f1_score(actual_labels, predicted_labels):
    return f1_score(actual_labels, predicted_labels, average="micro")

# Ensure actual_val is correctly defined as a NumPy array of true labels
actual_val = np.concatenate([y for x, y in test_tokenized_tf], axis=0)

# Evaluate bert trained model
f1_score_value = bert_f1_score(actual_val, preds_val)
print(f1_score_value)

0.61


# **Prompt Engineering**

In [ ]:
# Installation for GPU llama_cpp_python==0.2.28
!pip install llama-cpp-python==0.2.28
# For downloading the models from HF Hub huggingface-hub==0.23.2
!pip install huggingface-hub==0.23.2
# install evaluate==0.4.1 and bert-score==0.3.13 using pip command
!pip install evaluate==0.4.1 bert-score==0.3.13

# install numpy==1.25.2
!pip install numpy==1.25.2

In [ ]:
# Basic Imports for Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import re

import torch
import evaluate

from google.colab import drive
import locale

###**Importing Libaries and Setting up Mistral Model**

https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/blob/main/mistral-7b-instruct-v0.2.Q5_K_M.gguf

In [ ]:
## Import Hf_hub_download from hugging_face_hub
## Import Llama from llama_cpp
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [ ]:
# Define the model name or path as a string (You can find this info from hugging face website) Use Mistral

model_name_or_path = "TheBloke/Mistral-7B-Instruct-GGUF"

# Define the model basename as a string, indicating it's in the gguf format

model_basename = "mistral-7b-instruct-v0.2.05_K_M.gguf" # the model is in gguf format

In [1]:
import os
os.environ['HF_TOKEN'] = 'Token_placeholder'
print(os.getenv('HF_TOKEN'))

Token_placeholder


In [ ]:
model_path = hf_hub_download(
    repo_id="TheBloke/Mistral-7B-Instruct-v0.2-GGUF",
    filename="mistral-7b-instruct-v0.2.Q5_K_M.gguf",
    use_auth_token=os.getenv('HF_TOKEN')
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q5_K_M.gguf:   0%|          | 0.00/5.13G [00:00<?, ?B/s]

In [ ]:
# Create an instance of the 'Llama' class with specified parameters
# remove the blank spaces and complete the code

lcpp_llm = Llama(
        model_path=model_path,
        n_threads=8,  # CPU cores
        n_batch=512,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
        n_gpu_layers=20,  # Change this value based on your model and your GPU VRAM pool.
        n_ctx=2048,  # Context window
    )

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 1 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


#**Zero-Shot Prompting**

In [ ]:
system_message =  "You are a helpful assistant. Please generate a polite response to the user's complaint."

In [ ]:
zero_shot_prompt_template = "{system_message}\n\nCustomer Complaint: {user_input}\nProduct Category: "

In [ ]:
# Define function that combines user_prompt and system_message to create the prompt
#def generate_prompt(system_message,user_input):
#    prompt = zero_shot_prompt_template.format(system_message=system_message, user_input=user_input)
#    return prompt
def generate_prompt(system_message, user_input):
    prompt = f"{system_message}\n\nCustomer Complaint: {user_input}\nProduct Category: "
    return prompt

In [ ]:
def generate_mistral_response(input_text):

    # Combine user_prompt and system_message to create the prompt
    prompt = generate_prompt(system_message, input_text)

    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1200,
        temperature=0.0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=["/s"],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"].strip()  # Fill in the blank with .strip() to remove any extra spaces
    return response_text

**Due to limited GPU resources, model test with zero prompts on only 50 examples instead of the entire dataset.**

In [ ]:
# Randomly select 50 rows
new_data = data.sample(n=50, random_state=40)

In [ ]:
# example - new_data['mistral_response'] = new_data['narrative'].apply(lambda x:______ )
new_data['mistral_response'] = new_data['narrative'].apply(lambda x: generate_mistral_response(x))

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.gene

In [ ]:
def extract_category(text):
    # Define the regex pattern to match "category:" or "Category:" followed by a word
    pattern = r'category:\s*(\w+)'  # The pattern itself remains the same

    # Use re.search with the re.IGNORECASE flag to make it case-insensitive
    match = re.search(pattern, text, re.IGNORECASE)

    # If a match is found, return the captured group, else return None
    if match:
        return match.group(1)
    else:
        pattern1 = r'(credit_card|retail_banking|credit_reporting|mortgages_and_loans|debt_collection)'
        match = re.search(pattern1, text, re.IGNORECASE)
        if match:
            return match.group()
        else:
            return ''

#import re

#def extract_category(text):
#    # Define the regex pattern to match "category:" or "Category:" followed by a word
#    pattern = r'category:\s*(\w+)'  # The pattern itself remains the same

#    # Use re.search with the re.IGNORECASE flag to make it case-insensitive
 #   match = re.search(pattern, text, re.IGNORECASE)

 #   # If a match is found, return the captured group, else return None
 #   if match:
  #      return match.group(1)
  #  else:
   #     # Define another pattern to match specific categories
  #      pattern1 = r'(credit_card|retail_banking|credit_reporting|mortgages_and_loans|debt_collection)'
  #      match = re.search(pattern1, text, re.IGNORECASE)
  #      if match:
  #          return match.group()
  #      else:
  #          return ''

In [ ]:
new_data['mistral_response'] = new_data['mistral_response'].apply(lambda x: extract_category(x))

In [ ]:
# example - new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x:______ )
new_data['mistral_response_cleaned'] = new_data['narrative'].apply(lambda x: extract_category(x))

### **Calculate the F1 score**

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
###
# Test the function with a single example
test_narrative = new_data['narrative'].iloc[0]
test_response = generate_mistral_response(test_narrative)
print(f"Test Narrative: {test_narrative}")
print(f"Generated Response: {test_response}")

Llama.generate: prefix-match hit


Test Narrative: fraudulent charge totaling made capital one checking account via debit card seeing charge immediately canceled debit card informed capital one fraudulent charge disputed charge posted account issued provisional credit pending determination claim received form letter bank denying claim money withdrawn account opened appeal denial capital one bank person every contacted discus original claim make determination furthermore never lost possession debit card authorized person use capital one sent new debit card authorized via usps replace debit card set expire card sent fraudulent charge occurred never received card believe card intercepted fraudulently activated used make fraudulent purchase complained activated card several time capital one complained fraudulent charge last call dispute denial claim never noted account asked several time person making determination claim contact person making complaint malfeasance gross business practice sending replacement card without aut

In [ ]:
# Calculate F1 score for 'product' and 'mistral_response'
f1 = f1_score(new_data['product'], new_data['mistral_response'], average='macro')
f1 = f1_score(new_data['product'], new_data['mistral_response'], average='weighted')

print(f'F1 Score: {f1}')

F1 Score: 0.0


In [ ]:
# Calculate F1 score for 'product' and 'mistral_response_cleaned'
f2 =  f1_score(new_data['product'], new_data['mistral_response_cleaned'], average='macro')
f2 = f1_score(new_data['product'], new_data['mistral_response_cleaned'], average='weighted')
print(f'F1 Score: {f2}')

F1 Score: 0.0


#**Few-Shot Prompting**

**Generate a set of examples by randomly selecting 10 instances of user_input and assistant_output from dataset ensuring a balanced representation with 2 examples from each class.**

In [ ]:
# Separate categories
import json
review_1 = data[data['product'] == 'credit_card']
review_2 = data[data['product'] == 'retail_banking']
review_3 = data[data['product'] == 'credit_reporting']
review_4 = data[data['product'] == 'mortgages_and_loans']
review_5 = data[data['product'] == 'debt_collection']

# Sample 2 examples for each category
examples_1 = review_1.sample(2, random_state=40)
examples_2 = review_2.sample(2, random_state=40)
examples_3 = review_3.sample(2, random_state=40)
examples_4 = review_4.sample(2, random_state=40)
examples_5 = review_5.sample(2, random_state=40)

# Concatenate examples for few shot prompting
examples_df = pd.concat([examples_1,examples_2,examples_3,examples_4,examples_5 ])

# Create the gold examples for evaluation set by excluding examples
gold_examples_df = data.drop(index=examples_df.index)

# Convert examples to JSON
columns_to_select = ['narrative', 'product']
examples_json = examples_df[columns_to_select].to_json(orient='records')

# Print the first record from the JSON
print(json.loads(examples_json)[0])

# Print the shapes of the datasets
print("Examples Set Shape:", examples_df.shape)
print("Gold Examples Shape:", gold_examples_df.shape)

{'narrative': 'called request new york state covid relief plan day interest fee waived amex provided relief leading late payment amex refused honor relief day gap insists charging late fee', 'product': 'credit_card'}
Examples Set Shape: (10, 3)
Gold Examples Shape: (490, 3)


In [ ]:
system_message = "Please classify the following customer complaint into the appropriate category."

In [ ]:
first_turn_template = "User: {narrative}\nAssistant: The complaint falls under the category of {product}."
examples_template = "User: {narrative}\nAssistant: The complaint falls under the category of {product}."
prediction_template = "User: {narrative}\nAssistant: The complaint falls under the category of {product}."

In [ ]:
def generate_few_shot_prompt(system_message, examples_df, n=3):

    examples = []

    for i in range(min(n, len(examples_df))):
        example = examples_df.iloc[i]
        examples.append(f"User: {example['narrative']}\nAssistant: The complaint falls under the category of {example['product']}.")

    prompt = system_message + "\n\n" + "\n\n".join(examples)
    return prompt

In [ ]:
few_shot_prompt = generate_few_shot_prompt(system_message, examples_df)

In [ ]:
def create_few_shot_prompt(system_message, examples):

    """
    Return a prompt message in the format expected by Mistral 7b.
    10 examples are selected randomly as golden examples to form the
    few-shot prompt.
    We then loop through each example and parse the narrative as the user message
    and the product as the assistant message.

    Args:
        system_message (str): system message with instructions for classification
        examples(DataFrame): A DataFrame with examples (product + narrative + summary)
        to form the few-shot prompt.

    Output:
        few_shot_prompt (str): A prompt string in the Mistral format
    """

    few_shot_prompt = ''

    # Specify the columns to select from the examples DataFrame
    columns_to_select = ["narrative", "product"]

    # Convert the selected columns to JSON format
    examples_json = (
        examples_df.loc[:, columns_to_select].to_json(orient='records')
    )

    for idx, example in enumerate(json.loads(examples_json)):
        user_input_example = example["narrative"]
        assistant_output_example = example["product"]

        if idx == 0:
            few_shot_prompt += mistral_first_turn_template.format(
                system_message=system_message,
                user_message=user_input_example,
                assistant_message=assistant_output_example
            )
        else:
            few_shot_prompt += mistral_examples_template.format(
                user_message=user_input_example,
                assistant_message=assistant_output_example
            )

    return few_shot_prompt

In [ ]:
import pandas as pd
import json

# Convert the JSON string to a DataFrame
examples_df = pd.DataFrame(json.loads(examples_json))

# Now, you can use the DataFrame with your function
few_shot_prompt = generate_few_shot_prompt(system_message, examples_df)

In [ ]:
#few_shot_prompt = generate_few_shot_prompt(examples_json, system_message)
few_shot_prompt = generate_few_shot_prompt(system_message, examples_df)

In [ ]:
def generate_prompt(few_shot_prompt,new_review):
    prompt =  few_shot_prompt + "\n" + new_review
    return prompt

In [ ]:
def generate_mistral_response(input_text):

    # Combine user_prompt and system_message to create the prompt
    prompt = system_message + "\n" + input_text

    # Generate a response from the LLaMA model
    response = lcpp_llm(
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]  ### Fill in the blank
    return response_text

In [ ]:
# Randomly select 50 rows
new_data = gold_examples_df.sample(n=50, random_state=40)

In [ ]:
from llama_cpp import Llama

# Ensure model_path is correctly defined
model_path = hf_hub_download(
    repo_id="TheBloke/Mistral-7B-Instruct-v0.2-GGUF",
    filename="mistral-7b-instruct-v0.2.Q5_K_M.gguf",
    use_auth_token=os.getenv('HF_TOKEN')
)

# Initialize the Llama model with model_path
llama = Llama(
    model_path=model_path,
    n_threads=4,
    n_batch=512,
    f16_kv=True,
    use_mlock=True
)

# Define the function that uses the model
def generate_mistral_response(input_text):
     # Truncate input_text to ensure the total tokens fit within the context window of 512
    max_input_length = 400  # Adjust this based on the length of other components in your prompt
    truncated_input = input_text[:max_input_length]

    prompt = f"User: {truncated_input}\nAssistant:"
    response = llama(prompt=prompt)
    response_text = response["choices"][0]["text"]
    return response_text

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 1 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
new_data['mistral_response'] = new_data['narrative'].apply(lambda x: generate_mistral_response(x))

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.gene

In [ ]:
new_data['mistral_response_cleaned'] = new_data['mistral_response'].apply(lambda x: extract_category(x))

###**Calculate F1 score**

In [ ]:
# Calculate F1 score for 'product' and 'mistral_response_cleaned'
f3 =  f1_score(new_data['product'], new_data['mistral_response_cleaned'], average='macro')
print(f'F1 Score: {f3}')

F1 Score: 0.0


##**Text to Text generation**

In [ ]:
system_message = "Please classify the following customer complaint into the appropriate product category."

In [ ]:
zero_shot_prompt_template = "{system_message}\n\nCustomer Complaint: {user_input}\nProduct Category: "

In [ ]:
# Define function that combines user_prompt and system_message to create the prompt
def generate_prompt(system_message,user_input):
    prompt = f"{system_message}\n\nCustomer Complaint: {user_input}\nProduct Category: "
    return prompt

In [ ]:
def generate_mistral_response(input_text):

    # Combine user_prompt and system_message to create the prompt
    prompt = generate_prompt(system_message, input_text)

    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1200,
        temperature=0.0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=["/s"],
        echo=False
    )

    # Extract and return the response text
    response_text = response["choices"][0]["text"]  ### Fill in the blank
    return response_text

### **Generate mistral_response column containing LLM generated summaries**

In [ ]:
# Randomly select 50 rows
gold_examples = data.sample(n=50, random_state=40)

In [ ]:
gold_examples['mistral_response'] = gold_examples['narrative'].apply(generate_mistral_response)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.gene

### **Evaluate bert score**

In [ ]:
def evaluate_score(result, scorer, bert_score=False):

    """
    Return the ROUGE score or BERTScore for predictions on gold examples
    For each example we make a prediction using the prompt.
    Gold summaries and the AI generated summaries are aggregated into lists.
    These lists are used by the corresponding scorers to compute metrics.
    Since BERTScore is computed for each candidate-reference pair, we take the
    average F1 score across the gold examples.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        scorer (function): Scorer function used to compute the ROUGE score or the
                           BERTScore
        bert_score (boolean): A flag variable that indicates if BERTScore should
                              be used as the metric.

    Output:
        score (float): BERTScore or ROUGE score computed by comparing model predictions
                       with ground truth
    """

    model_predictions = result['mistral_response'].tolist()
    ground_truths = result['summary'].tolist()

    if bert_score:
        score = scorer.compute(
            predictions=model_predictions,
            references=ground_truths,
            lang="en",
            rescale_with_baseline=True
        )
        return sum(score['f1']) / len(score['f1'])
    else:
        return scorer.compute(
            predictions=model_predictions,
            references=ground_truths
        )

In [ ]:
bert_scorer = evaluate.load("bertscore")

In [ ]:
score = evaluate_score(gold_examples, bert_scorer, bert_score=True)
print(f'BERTScore: {score}')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: 0.04154061934445053


### **Observations**

The BERTScore achieved is 0.0415, which is relatively low. This indicates that the generated responses may not align well with the gold standard. The low score suggests that the model might need further fine-tuning or that the generated responses are significantly different in meaning or structure from the reference responses.